In [2]:
import os
import pandas as pd
import numpy as np
from os.path import *

## Load data
- load csv and convert to numpy

In [78]:
%load_ext autoreload
%autoreload 2
from converter import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
label = pd.read_csv('1_label.csv')
neuro = pd.read_csv('1_neuro.csv')

In [80]:
start = 1548120820000
end = 1548121672000
cols = ['delta', 'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta', 'lowGamma', 'middleGamma', 'theta']
data = match_ts(start, end, neuro, cols)

In [81]:
data.shape

(170, 5, 8)

In [7]:
np.save('sample.npy',data)

In [8]:
np.load('sample.npy').shape

(170, 5, 8)

In [86]:
y.shape

(181, 3)

In [88]:
y = onehot_label(label, 'arousal')
y.shape

y_ =[]

for x in y:
    if x[0] == 1:
        y_.append(0)
        continue
    if x[1] == 1:
        y_.append(1)
        continue
    if x[2] == 1:
        y_.append(2)
        continue
         
    y_.append(-1)


/home/doomguy/projects/02_SDS_PROJ/converter.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label['l'] = label[col]<3.0
/home/doomguy/projects/02_SDS_PROJ/converter.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label['m'] = label[col]==3.0
/home/doomguy/projects/02_SDS_PROJ/converter.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [91]:
y= np.array(y_)

## Normalize & Split

In [77]:
data= data.astype('float32')

In [92]:
from sklearn.model_selection import train_test_split

data = (data- data.min())/(data.max() - data.min())

X_train, X_test, y_train, y_test = train_test_split(data, y[:170], random_state=0)
X_train.shape, X_test.shape

((127, 5, 8), (43, 5, 8))

In [93]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model


In [94]:
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

In [95]:
X_train.shape

(127, 5, 8, 1)

In [96]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [97]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(3, 1, activation='relu')
    self.conv2 = Conv2D(2, 3, activation='relu')
    self.conv3= Conv2D(1, 1, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(62, activation='relu')
    self.d2 = Dense(3, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.flatten(x)
    print(x.shape)
    x = self.d1(x)
    print(x.shape)
    return self.d2(x)

model = MyModel()

In [98]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [99]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(32, 18)
(32, 62)
(32, 18)
(32, 62)
(31, 18)
(31, 62)
(32, 18)
(32, 62)
(11, 18)
(11, 62)
에포크: 1, 손실: 1.091733694076538, 정확도: 74.80314636230469, 테스트 손실: 1.0796475410461426, 테스트 정확도: 86.0465087890625
에포크: 2, 손실: 1.0818649530410767, 정확도: 79.52755737304688, 테스트 손실: 1.0678019523620605, 테스트 정확도: 86.0465087890625
에포크: 3, 손실: 1.0699740648269653, 정확도: 81.10235595703125, 테스트 손실: 1.0543012619018555, 테스트 정확도: 86.0465087890625
에포크: 4, 손실: 1.056455135345459, 정확도: 81.88976287841797, 테스트 손실: 1.0389115810394287, 테스트 정확

In [105]:
list(y_train).count(1)/len(y_train)

0.84251968503937

In [106]:
list(y_test).count(1)/len(y_test)

0.8604651162790697